## Import modules

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
import json
import os
import math
import requests
import geometry
from PIL import Image

## Set input and output folders

In [ ]:
# input_folder='data'
input_folder='output\Wagga\Panos'
input_folder_depth='output\Wagga\Depth_decoded'

out_folder='output\Wagga\Panos_clipped'
out_folder_depth='output\Wagga\Depth_decoded_clipped'

## Load metadata file

In [ ]:
# Read the JSON file
metadata_file=os.path.join(input_folder,'metadata_all.json')
with open(metadata_file, 'r') as json_file:
    metadata = json.load(json_file)
pano_ids = list(metadata.keys())
len(pano_ids)

In [ ]:
# pano_id=pano_ids[2]
pano_id='8z3aOgFCC5fbk1vvH3RWuQ'
pano_id

## Read in pano

In [ ]:
pano_file=os.path.join(input_folder,pano_id+".jpg")
gsv_pano=Image.open(pano_file)
gsv_pano

## Read in decoded depth map

In [ ]:
depth_file=os.path.join(input_folder_depth,pano_id+".tif")
gsv_depth=Image.open(depth_file)
gsv_depth

## Localise building of interest

House location (horizontal pixel range) within pano image:

In [ ]:
# below are standard google pano image dimension which does not apply to user uploaded data
# width=512*(2**zoom_level) # pano image width
# height=512*(2**(zoom_level-1)) # pano image height
heading=metadata[pano_id]['All_panos'][pano_id]['heading']
location = geometry.localize_house_in_panorama(lat_c=metadata[pano_id]['location']['lat'], 
                                    lon_c=metadata[pano_id]['location']['lng'], 
                                    lat_house=metadata[pano_id]['house_location']['lat'], 
                                    lon_house=metadata[pano_id]['house_location']['lng'],
                                    beta_yaw_deg=heading, 
                                    Wim=gsv_pano.width,angle_extend=40)
location

## Clip panorama image to localised area
- Horizontally clipped to calculated pixel range
- Vertically clipped based on hard-coded upper and lower crop proportions range

In [ ]:
upper_crop=0.25
lower_crop=0.6

In [ ]:
gsv_pano_clipped=gsv_pano.crop(box=(location['horizontal_pixel_range_house'][0],
                   upper_crop*gsv_pano.height,
                   location['horizontal_pixel_range_house'][1],
                   lower_crop*gsv_pano.height
                   ))
gsv_pano_clipped

In [ ]:
gsv_depth_clipped=gsv_depth.crop(box=(location['horizontal_pixel_range_house'][0],
                   upper_crop*gsv_pano.height,
                   location['horizontal_pixel_range_house'][1],
                   lower_crop*gsv_pano.height
                   ))
gsv_depth_clipped

### Save clipped images

In [ ]:
gsv_pano_clipped.save(os.path.join(out_folder,pano_id+".jpg"), "jpeg")

In [ ]:
gsv_depth_clipped.save(os.path.join(out_folder_depth,pano_id+".tif"))

## Batch clipping

In [ ]:
for pano_id in pano_ids:
    if metadata[pano_id].get('status') == 'OK':
        try:
            # read in pano image
            pano_file=os.path.join(input_folder,pano_id+".jpg")
            gsv_pano=Image.open(pano_file)

            # read in depth map
            depth_file=os.path.join(input_folder_depth,pano_id+".tif")
            gsv_depth=Image.open(depth_file)
            
            # localise house of interest
            heading=metadata[pano_id]['All_panos'][pano_id]['heading']
            location = geometry.localize_house_in_panorama(lat_c=metadata[pano_id]['location']['lat'], 
                                            lon_c=metadata[pano_id]['location']['lng'], 
                                            lat_house=metadata[pano_id]['house_location']['lat'], 
                                            lon_house=metadata[pano_id]['house_location']['lng'],
                                            beta_yaw_deg=heading,
                                            Wim=gsv_pano.width,angle_extend=40)
            # clip pano image
            gsv_pano_clipped=gsv_pano.crop(box=(location['horizontal_pixel_range_house'][0],
                        upper_crop*gsv_pano.height,
                        location['horizontal_pixel_range_house'][1],
                        lower_crop*gsv_pano.height
                        ))
            
            # clip depth map
            gsv_depth_clipped=gsv_depth.crop(box=(location['horizontal_pixel_range_house'][0],
                   upper_crop*gsv_pano.height,
                   location['horizontal_pixel_range_house'][1],
                   lower_crop*gsv_pano.height
                   ))
            # save files
            gsv_pano_clipped.save(os.path.join(out_folder,pano_id+".jpg"), "jpeg")
            gsv_depth_clipped.save(os.path.join(out_folder_depth,pano_id+".tif"))
        except Exception as e:  # Catch any other unexpected errors:
            print(f"{e}, skipping...")